# Initialize

## Libraries

In [ ]:
# For loading finetuned model
from modules.train.trainer import get_model_tokenizer

# For testing performance
from modules.prediction.predict import run_prediction

# For loading test dataset
from modules.data.hfdata import load_imdb, load_squad, load_wmt

# For saving
import json
import torch

## Configuration

In [ ]:
# List of implemented methods
models   = ['t5-base', 'bart-base', 'prophetnet-large-uncased']
datasets = ['squad', 'wmt16_en_de', 'imdb']
finetunes = ['full', 'lora', 'adapters']

# Selecting index
model, dataset, finetune = 0, 0, 0

In [ ]:
task = {
    "squad": "qa",
    "wmt16_en_de": "translation",
    "imdb": "textsentiment"
}

model_path = f'models/ft-{models[model]}-{finetunes[finetune]}-{task[datasets[dataset]]}'

Device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}.')

# Loading Model & Tokenizer

In [ ]:
# Load finetuned model
model, tokenizer = get_model_tokenizer(model_path, device)

# Loading Dataset

Getting data config

In [ ]:
# get config from file
with open('modules/data/config.json', 'r', encoding='utf-8') as file:
    data_config = json.load(file)

In [ ]:
test_dataset = None

if dataset == 0:
    _, test_dataset, _ = load_squad(test=False, data_config=data_config)
    test_dataset = [
        {
            "input": "question: " + q + " context: " + c,
            "target": a["text"][0]
        }
        for q, c, a in zip(test_dataset["question"], test_dataset["context"], test_dataset["answers"])
    ]

elif dataset == 1:
    _, test_dataset, _ = load_wmt(test=False, data_config=data_config)
    test_dataset = [
        {
            "input": d['translation']['en'],
            "target": d['translation']['de']
        }
        for d in test_dataset
    ]
else: # dataset == 2
    _, test_dataset, _ = load_imdb(test=False, data_config=data_config)
    test_dataset = [
        {
            "input": d['test'],
            "target": "positive" if d['label'] == 1 else "negative"
        }
        for d in test_dataset
    ]

# Run Evaluation

In [ ]:
output_dir = f'prediction/prediction-{models[model]}-{finetunes[finetune]}-{task[datasets[dataset]]}.json'

success = run_prediction(model=model, tokenizer=tokenizer, test_dataset=test_dataset, output_dir=output_dir)

if success:
    print(f'Predicted values stored in {output_dir}')
else:
    print('Error in predicting.')